In [27]:
import sqlite3
import json

In [35]:
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

conn = sqlite3.connect('programs.db', isolation_level=None)
conn.row_factory = dict_factory
cursor = conn.cursor()

In [10]:
def get_bachelors_programs_json(): 
    """
    Выдает словарь, где ключами являются названия образовательных программ, 
    а значениями — словари с признаками для каждой программы.
    """
    import re
    import requests
    from bs4 import BeautifulSoup
    import json
    link = "https://www.hse.ru/education/programs"
    r = requests.get(link)
    page = BeautifulSoup(r.text, 'html.parser')
    data_programs_bac_json = {}
    
    for program__title in page.body.find_all("div", re.compile("education_bachelor")):
        field = program__title.h3.text #направление подготовки 07.00.00 Архитектура

        for item in program__title.find_all("div", class_="edu-programm__item small"):
            program = str(item.a.text) # название образовательной программы
            campus = item.find("div", class_="edu-programm__campus").text
            faculty = item.find("div", class_="edu-programm__unit").text #факультет
            duration = item.find("div", class_="edu-programm__edu").text.split()[0] #продолжительность обучения
            form = item.find("span", class_="edu-programm__edu_offline").text #форма обучения - Очная\Заочная
            link = item.find("a").get("href") 

            data_programs_bac_json[program] = {
                "campus": campus,
                "faculty": faculty,
                "duration": duration,
                "form": form,
                "link": link,
                "field": field
            }

            # Этот блок обрабатывает количество мест.

            openings_free_list = item.find_all("div", "edu-programm__place_free") 
            openings_free_list = [n.text.strip().replace("\xa0", " ") for n in openings_free_list]
            if len(openings_free_list) > 1:
                openings_free_sum = int(openings_free_list[0].split()[0])+int(openings_free_list[-1].split()[0])
            else:
                try:
                    openings_free_sum = int(openings_free_list[0].split()[0])
                except IndexError: 
                    openings_free_sum = 0

            openings_paid = item.find_all("div", class_="edu-programm__place_paid")
            openings_paid = [n.text.strip().replace("\xa0", " ") for n in openings_paid]
            if len(openings_paid) > 1:
                openings_paid_rus = int(openings_paid[0].split()[0])
                openings_paid_foreign = int(openings_paid[-1].split()[0])
                data_programs_bac_json[program]["openings"] = {
                    "free": openings_free_sum,
                    "paid_rus": openings_paid_rus,
                    "paid_foreign": openings_paid_foreign}
            else:
                openings_paid_rus = int(openings_paid[0].split()[0])
                data_programs_bac_json[program]["openings"] = {
                    "free": openings_free_sum,
                    "paid_rus": openings_paid_rus}

    return data_programs_bac_json

In [11]:
programs = get_bachelors_programs_json()
list(programs.items())[:3]

[('Библеистика и история древнего Израиля',
  {'campus': 'Москва',
   'duration': '5',
   'faculty': 'Институт классического Востока и античности',
   'field': '58.00.00 Востоковедение и африканистика',
   'form': 'Очная',
   'link': 'https://www.hse.ru/ba/israel/',
   'openings': {'free': 12, 'paid_foreign': 5, 'paid_rus': 10}}),
 ('Востоковедение',
  {'campus': 'Санкт-Петербург',
   'duration': '5',
   'faculty': 'Санкт-Петербургская школа социальных и гуманитарных наук',
   'field': '41.00.00 Политические науки и регионоведение',
   'form': 'Очная',
   'link': 'https://spb.hse.ru/ba/oriental/',
   'openings': {'free': 25, 'paid_foreign': 4, 'paid_rus': 70}}),
 ('Языки и литература Индии',
  {'campus': 'Москва',
   'duration': '5',
   'faculty': 'Институт классического Востока и античности',
   'field': '58.00.00 Востоковедение и африканистика',
   'form': 'Очная',
   'link': 'https://www.hse.ru/ba/india/',
   'openings': {'free': 12, 'paid_foreign': 5, 'paid_rus': 10}})]

In [12]:
new_programs = []
for k, v in programs.items():
    v['field'] = v['field'].split(maxsplit=1)
    v['name'] = k
    new_programs.append(v)
programs = new_programs
del new_programs
programs[:3]

[{'campus': 'Москва',
  'duration': '5',
  'faculty': 'Институт классического Востока и античности',
  'field': ['58.00.00', 'Востоковедение и африканистика'],
  'form': 'Очная',
  'link': 'https://www.hse.ru/ba/israel/',
  'name': 'Библеистика и история древнего Израиля',
  'openings': {'free': 12, 'paid_foreign': 5, 'paid_rus': 10}},
 {'campus': 'Санкт-Петербург',
  'duration': '5',
  'faculty': 'Санкт-Петербургская школа социальных и гуманитарных наук',
  'field': ['41.00.00', 'Политические науки и регионоведение'],
  'form': 'Очная',
  'link': 'https://spb.hse.ru/ba/oriental/',
  'name': 'Востоковедение',
  'openings': {'free': 25, 'paid_foreign': 4, 'paid_rus': 70}},
 {'campus': 'Москва',
  'duration': '5',
  'faculty': 'Институт классического Востока и античности',
  'field': ['58.00.00', 'Востоковедение и африканистика'],
  'form': 'Очная',
  'link': 'https://www.hse.ru/ba/india/',
  'name': 'Языки и литература Индии',
  'openings': {'free': 12, 'paid_foreign': 5, 'paid_rus': 10

In [14]:
fields = list({tuple(program['field']) for program in programs})
forms = list({program['form'] for program in programs})
faculties = list({program['faculty'] for program in programs})
campuses = list({program['campus'] for program in programs})

In [15]:
cursor.execute('DROP TABLE IF EXISTS fields')
sql = '''CREATE TABLE fields (
    code text PRIMARY KEY,
    name text
)'''
cursor.execute(sql)

sql = """INSERT INTO fields (code, name) VALUES (?, ?)"""
for code, name in fields:
    cursor.execute(sql, [code, name])

In [17]:
sql = """SELECT * FROM fields WHERE code in ('41.00.00', '58.00.00')"""
for row in conn.execute(sql):
    print(row)

('41.00.00', 'Политические науки и регионоведение')
('58.00.00', 'Востоковедение и африканистика')


In [18]:
cursor.execute('DROP TABLE IF EXISTS faculties')
sql = '''CREATE TABLE faculties (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name text
)'''
cursor.execute(sql)

sql = """INSERT INTO faculties (name) VALUES (?)"""
for name in faculties:
    cursor.execute(sql, [name])

In [19]:
cursor.execute('DROP TABLE IF EXISTS forms')
sql = '''CREATE TABLE forms (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name text
)'''
cursor.execute(sql)

sql = """INSERT INTO forms (name) VALUES (?)"""
for name in forms:
    cursor.execute(sql, [name])

In [20]:
sql = '''CREATE TABLE campuses (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name text
)'''
cursor.execute(sql)

sql = """INSERT INTO campuses (name) VALUES (?)"""
for name in campuses:
    cursor.execute(sql, [name])

In [21]:
programs[0].keys()

dict_keys(['campus', 'faculty', 'duration', 'form', 'link', 'field', 'openings', 'name'])

In [22]:
{p['duration'] for p in programs}

{'4', '5', '5,5'}

In [25]:
cursor.execute('DROP TABLE IF EXISTS programs')
sql = '''CREATE TABLE programs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name text,
    link text,
    duration TEXT,
    openings TEXT,
    faculty_id INTEGER,
    campus_id INTEGER,
    form_id INT,
    field_code TEXT,
    
    FOREIGN KEY(faculty_id) REFERENCES faculties(id),
    FOREIGN KEY(campus_id) REFERENCES campuses(id),
    FOREIGN KEY(form_id) REFERENCES forms(id),
    FOREIGN KEY(field_code) REFERENCES fields(code)
)'''
cursor.execute(sql)

In [28]:
sql = """
INSERT INTO programs (name, link, duration, openings, faculty_id, campus_id, form_id, field_code)
SELECT :name, :link, :duration, :openings, f.id, c.id, forms.id, :field
FROM faculties f, campuses c, forms
WHERE f.name=:faculty AND c.name=:campus AND forms.name=:form
"""
for program in programs:
    p = dict(program)
    p['field'] = p['field'][0]
    p['openings'] = json.dumps(p['openings'], ensure_ascii=False)
    conn.execute(sql, p)

In [38]:
sql = """
SELECT p.*, f.name faculty, f.name field_name
FROM programs p
JOIN faculties f on f.id=p.faculty_id
JOIN fields fl on fl.code=p.field_code
WHERE f.name='Санкт-Петербургская школа социальных и гуманитарных наук'
"""
cursor.execute(sql)
cursor.fetchall()

[{'campus_id': 2,
  'duration': '5',
  'faculty': 'Санкт-Петербургская школа социальных и гуманитарных наук',
  'faculty_id': 15,
  'field_code': '41.00.00',
  'field_name': 'Санкт-Петербургская школа социальных и гуманитарных наук',
  'form_id': 1,
  'id': 2,
  'link': 'https://spb.hse.ru/ba/oriental/',
  'name': 'Востоковедение',
  'openings': '{"free": 25, "paid_rus": 70, "paid_foreign": 4}'},
 {'campus_id': 2,
  'duration': '4',
  'faculty': 'Санкт-Петербургская школа социальных и гуманитарных наук',
  'faculty_id': 15,
  'field_code': '41.00.00',
  'field_name': 'Санкт-Петербургская школа социальных и гуманитарных наук',
  'form_id': 1,
  'id': 21,
  'link': 'https://spb.hse.ru/ba/political/',
  'name': 'Политология и мировая политика',
  'openings': '{"free": 40, "paid_rus": 50, "paid_foreign": 10}'},
 {'campus_id': 2,
  'duration': '4',
  'faculty': 'Санкт-Петербургская школа социальных и гуманитарных наук',
  'faculty_id': 15,
  'field_code': '39.00.00',
  'field_name': 'Санкт-

In [39]:
cursor.execute('SELECT * FROM campuses')
cursor.fetchall()

[{'id': 1, 'name': 'Нижний Новгород'},
 {'id': 2, 'name': 'Санкт-Петербург'},
 {'id': 3, 'name': 'Пермь'},
 {'id': 4, 'name': 'Москва'}]